In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm import tqdm
import numpy as np
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pymupdf

## Get all law case links

In [2]:
# search queries
queries = ['forced labour',
            'forced labor',
            'human trafficking​',
            'child labor',
            'child labour',
            'modern slavery',
            'slavery']

# required header
header = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:128.0) Gecko/20100101 Firefox/128.0',
          'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/png,image/svg+xml,*/*;q=0.8'
          }

domain = 'http://www.liiofindia.org/'

# base query template
query_base = 'http://www.liiofindia.org/cgi-bin/sinosrch.cgi?method=boolean&query={}&meta=%2Fliiofindia&lii=LIIofIndia&mask_path=in%2Fcases'

In [21]:
def get_law_urls(soup):
    """
    Get all law case urls from search page
    """
    law_cases_url = []
    law_cases_list = soup.find('ol')

    # some queries return empty response
    if law_cases_list == None:
        return None
    
    law_cases_list = law_cases_list.find_all('li')
    
    for case in law_cases_list:
        for item in case.p.contents:
            if item.name == 'a':
                url_case = item['href']
                title = item.contents[0]
                year = [int(y) for y in re.findall(r'\d{4}', title)]
                year = min(year) if year != [] else None
            elif item.name == 'small':
                database = item.contents[1].contents[0]
        
        # check if its law case
        if re.search(r'\s[vV][sS]?\.?\s', title):
            law_cases_url.append({'url': url_case,
                                'title': title,
                                'year': year,
                                'database': database})
    return law_cases_url

In [25]:
law_cases = []
for query in queries:
    query_cases = []
    url = query_base.format(query.replace(' ', '+'))
    soup = BeautifulSoup(requests.get(url, headers=header).content, "html.parser")

    search_pages = []
    for url in soup.find_all('a'):
        if url.has_attr('href') and re.match(r'.*offset=\d+0.*', url['href']):
            search_pages.append(url['href'])
    search_pages = ['http://www.liiofindia.org' + page for page in list(set(search_pages)) if 'http://www.liiofindia.org' not in page]

    # search links in first page
    urls = get_law_urls(soup)
    if urls != None:
        query_cases += urls

        # search in offset pages
        for page in search_pages:
            soup = BeautifulSoup(requests.get(page, headers=header).content, "html.parser")
            query_cases += get_law_urls(soup)

        # update query value
        for idx, case in enumerate(query_cases):
            query_cases[idx]['query'] = query
        
        # add to law cases list
        law_cases += query_cases
    
    print(f'{query}: {len(query_cases)}')

forced labour: 201
forced labor: 1
human trafficking​: 0
child labor: 13
child labour: 240
modern slavery: 0
slavery: 248


In [ ]:
df = pd.DataFrame(law_cases)
df.to_csv('search.csv', index=False)

NameError: name 'law_cases' is not defined

In [96]:
df.head()

,url,title,year,database,query,content
0,http://www.liiofindia.org/cgi-bin/disp.pl/in/c...,PEOPLE'S UNION FOR DEMOCRATIC RIGHTS & ORS V U...,1982,High Court of Karnataka,forced labour,"BHAGWATI, P.N.\nBHAGWATI, P.N.\nISLAM, BAHARUL..."
1,http://www.liiofindia.org/cgi-bin/disp.pl/in/c...,PEOPLE'S UNION FOR DEMOCRATIC RIGHTS & ORS V U...,1473,Supreme Court of India,forced labour,NaN
2,http://www.liiofindia.org/cgi-bin/disp.pl/in/c...,PEOPLE'S UNION FOR DEMOCRATIC RIGHTS & ORS V U...,1473,Supreme Court of India,forced labour,NaN
3,http://www.liiofindia.org/cgi-bin/disp.pl/in/c...,BANDHUA MUKTI MORCHA V UNION OF INDIA & ORS [1...,1983,High Court of Karnataka,forced labour,NaN
4,http://www.liiofindia.org/cgi-bin/disp.pl/in/c...,BANDHUA MUKTI MORCHA V UNION OF INDIA & ORS [1...,1151,Supreme Court of India,forced labour,NaN


## Extract page content

In [ ]:
df = pd.read_csv('search.csv')
df['content'] = np.nan

In [ ]:
def get_content(soup):
    soup.find_all('h2')
    title = soup.h2
    document = title.find_next('p')

    # remove footnote
    document = re.sub(r'\[\nContext\n\] \[\nHide Context\n\]\n(?:CommonLII|LIIofIndia):\nCopyright Policy\n\|\nDisclaimers\n\|\nPrivacy Policy\n\|\nFeedback\nURL:\s.*', '', document.get_text(strip=True, separator='\n'))
    
    return document

In [ ]:
for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
    if not pd.isna(row.content): continue
    sleep(0.01)
    soup = BeautifulSoup(requests.get(row.url, headers=header).content, "html.parser")

    # check if it is embbeded pdf
    obj = soup.find('object')
    if obj is not None and obj['type'] == 'application/pdf':
        df.loc[idx, 'content'] = ''
        continue

    content = get_content(soup)
    df.loc[idx, 'content'] = content

100%|██████████| 703/703 [30:54<00:00,  2.64s/it]  


In [ ]:
df.to_csv('lii_india_modern_slavery.csv', index=False)

In [100]:
len(df.loc[df['content']==''])

283

## Download Embedded PDFs and extract their content

In [ ]:
df = pd.read_csv('lii_india_modern_slavery.csv')
pdf_df = df[df.isnull().any(axis=1)]
pdf_df.shape

(3, 6)

In [49]:
last_downloaded = -1 # checkpoint
data_path = './data/' # download path
downloaded_files = [] # list of downloaded files

In [50]:
for idx, row in tqdm(pdf_df.iterrows(), total=pdf_df.shape[0]):
    if idx <= last_downloaded: continue
    sleep(0.01)
    soup = BeautifulSoup(requests.get(row.url, headers=header).content, "html.parser")
    
    obj = soup.find('object')
    if obj is not None and obj['type'] == 'application/pdf':
        file_path = obj['data'] # get file location in database
        filename = file_path.replace('/', '_').strip('_').replace('in_cases_', '')
        sleep(0.01)
        content = requests.get(domain+file_path, headers=header).content # get file from database
        with open(data_path+filename, 'wb') as f:
            f.write(content)
        downloaded_files.append((idx, filename)) # save idx and filename
    elif 'There is no available HTML version of this document' in soup.get_text():
        if file_path := re.search(r'(in\/cases\/.*\.html)', row.url):
            file_path = file_path.group(1).replace('html', 'pdf')
            filename = file_path.replace('/', '_').strip('_').replace('in_cases_', '')
            sleep(0.01)
            content = requests.get(domain+file_path, headers=header).content # get file from database
            with open(data_path+filename, 'wb') as f:
                f.write(content)
            downloaded_files.append((idx, filename)) # save idx and filename
    else:
        downloaded_files.append((idx, -1)) # save idx and -1 if download file not found

    last_downloaded = idx

100%|██████████| 49/49 [13:26<00:00, 16.45s/it]


In [51]:
print(len(downloaded_files))

49


In [52]:
idx_list = [idx for idx, _ in downloaded_files]
doc_content = []
for idx, file in downloaded_files:
    if file == -1:
        doc_content.append('')
    else:
        doc = pymupdf.open(data_path+file)
        content = ' '.join([page.get_text() for page in doc])
        doc_content.append(content)

In [53]:
df.loc[idx_list, 'content'] = doc_content

In [ ]:
df.to_csv('lii_india_modern_slavery.csv', index=False)